# Super Store Analysis

---

## Installing dependencies

In [97]:
# !pip install xlrd
# !pip install pandas
# !pip install numpy
# !pip install mysql-connector-python
# !pip install sqlalchemy
# !pip install pymysql
# !pip install google-cloud-bigquery
# !pip install db-dtypes
# !pip install db-dtypes

---

## Setting up the environment

In [98]:
import re
import os
import warnings
import pandas as pd
import numpy as np
import mysql.connector as mysql
from sqlalchemy import create_engine, text
from sqlalchemy.exc import *
from google.cloud import bigquery as bq
from google.oauth2 import service_account

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # show max columns
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'explore29-33756158108f.json'

---

## Extract

In [99]:
file_path = 'data-src/Sample - Superstore.xls'
orders_df = pd.read_excel(file_path, sheet_name='Orders', header=0)
people_df = pd.read_excel(file_path, sheet_name='People', header=0)
returns_df = pd.read_excel(file_path, sheet_name='Returns', header=0)

---

## Transform

### Orders

#### Overview

In [100]:
orders_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State/Province,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,Pennsylvania,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [101]:
orders_df.tail()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State/Province,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
10189,10190,US-2024-143259,2024-12-30,2025-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,New York,10009,East,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,52.776,3,0.2,19.7910
10190,10191,US-2024-115427,2024-12-30,2025-01-03,Standard Class,EB-13975,Erica Bern,Corporate,United States,Fairfield,California,94533,West,OFF-BI-10004632,Office Supplies,Binders,GBC Binding covers,20.720,2,0.2,6.4750
10191,10192,US-2024-156720,2024-12-30,2025-01-03,Standard Class,JM-15580,Jill Matthias,Consumer,United States,Loveland,Colorado,80538,West,OFF-FA-10003472,Office Supplies,Fasteners,Bagged Rubber Bands,3.024,3,0.2,-0.6048
10192,10193,US-2024-143259,2024-12-30,2025-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,New York,10009,East,TEC-PH-10004774,Technology,Phones,Gear Head AU3700S Headset,90.930,7,0.0,2.7279
10193,10194,CA-2024-143500,2024-12-30,2025-01-03,Standard Class,HO-15230,Harry Olson,Consumer,Canada,Charlottetown,Prince Edward Island,C0A,East,OFF-BI-10004040,Office Supplies,Binders,Wilson Jones Impact Binders,3.024,3,0.2,-0.6048


In [102]:
orders_df.shape

(10194, 21)

In [103]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          10194 non-null  int64         
 1   Order ID        10194 non-null  object        
 2   Order Date      10194 non-null  datetime64[ns]
 3   Ship Date       10194 non-null  datetime64[ns]
 4   Ship Mode       10194 non-null  object        
 5   Customer ID     10194 non-null  object        
 6   Customer Name   10194 non-null  object        
 7   Segment         10194 non-null  object        
 8   Country/Region  10194 non-null  object        
 9   City            10194 non-null  object        
 10  State/Province  10194 non-null  object        
 11  Postal Code     10194 non-null  object        
 12  Region          10194 non-null  object        
 13  Product ID      10194 non-null  object        
 14  Category        10194 non-null  object        
 15  Su

---

#### Renaming headers

In [104]:
def snake_case(data:str) -> str:
	return(data.lower().strip().replace(' ','_').replace('-', '_'))

In [105]:
orders_df = orders_df.rename(columns={
	'Country/Region': 'country',
	'State/Province': 'province',
	'Postal Code': 'post_code'
})

orders_df.columns = [snake_case(col) for col in orders_df.columns]

---

#### Cleaning

**Check for null values**

In [106]:
print(orders_df.isnull().sum())

row_id           0
order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
customer_name    0
segment          0
country          0
city             0
province         0
post_code        0
region           0
product_id       0
category         0
sub_category     0
product_name     0
sales            0
quantity         0
discount         0
profit           0
dtype: int64


---

**Check for duplicates**

In [107]:
print(orders_df.duplicated(subset=['order_id', 'product_id'], keep='last').sum())

11


**Show duplicates**

In [108]:
orders_df[orders_df.duplicated(subset=['order_id', 'product_id'], keep=False)].head(4)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
390,391,US-2021-150119,2021-04-23,2021-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
391,392,US-2021-150119,2021-04-23,2021-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
1698,1699,CA-2021-153623,2021-11-24,2021-12-05,Standard Class,JP-11135,James Peterman,Corporate,Canada,St. John's,Newfoundland and Labrador,A0A,East,FUR-FU-10002501,Furniture,Furnishings,Nu-Dell Executive Frame,99.120,8,0.0,35.4144
1699,1700,CA-2021-153623,2021-11-24,2021-12-05,Standard Class,JP-11135,James Peterman,Corporate,Canada,St. John's,Newfoundland and Labrador,A0A,East,FUR-FU-10002501,Furniture,Furnishings,Nu-Dell Executive Frame,99.120,8,0.0,35.4144


In [109]:
orders_df[orders_df.duplicated(subset=['order_id', 'product_id'], keep=False)].tail(4)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
9384,9385,US-2024-152912,2024-11-09,2024-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,1633.140,9,0.0,473.6106
9385,9386,US-2024-152912,2024-11-09,2024-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,544.380,3,0.0,157.8702
9800,9801,US-2024-118017,2024-12-03,2024-12-06,Second Class,LC-16870,Lena Cacioppo,Consumer,United States,Thornton,Colorado,80229,West,TEC-AC-10002006,Technology,Accessories,Memorex Micro Travel Drive 16 GB,76.752,6,0.2,10.5534
9801,9802,US-2024-118017,2024-12-03,2024-12-06,Second Class,LC-16870,Lena Cacioppo,Consumer,United States,Thornton,Colorado,80229,West,TEC-AC-10002006,Technology,Accessories,Memorex Micro Travel Drive 16 GB,102.336,8,0.2,14.0712


**Problem:**

Upon closer inspection, it is found that the duplicate columns have the same/different ```quantity```, which affects the ```sales``` and ```profit``` columns. Hence, identical rows with the same ```order_id``` and ```product_id``` on the same date, with same/different ```quantity``` will be merged.

**Aggregation to solve for duplicates:**

<details>
<summary>Aggregation breakdown</summary>

```py
orders_df = orders_df.groupby(['order_id', 'product_id'], as_index=False).agg({
	'row_id': 'last',
	'order_id': 'last',
	'order_date': 'last',
	'ship_date': 'last',
	'ship_mode': 'last',
	'customer_id': 'last',
	'customer_name': 'last',
	'segment': 'last',
	'country': 'last',
	'city': 'last',
	'province': 'last',
	'post_code': 'last',
	'region': 'last',
	'product_id': 'last',
	'category': 'last',
	'sub_category': 'last',
	'product_name': 'last',
	'discount': 'last',
	'sales': 'sum',
	'quantity': 'sum',
	'profit': 'sum'
})
```
</details>

In [110]:
# orders_df.drop_duplicates(keep='last')

# aggregation instructions: map to 'last' if not ['sales', 'quantity', 'profit'], else map 'sum'
agg_instruction = {col: ('sum' if col in ['sales', 'quantity', 'profit'] else 'last') for col in list(orders_df.columns)}
orders_df = orders_df.groupby(['order_id', 'product_id'], as_index=False).agg(agg_instruction)

orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1380,CA-2021-100867,2021-10-19,2021-10-24,Standard Class,EH-14125,Eugene Hildebrand,Home Office,Canada,Vancouver,British Columbia,V6G,West,TEC-PH-10004922,Technology,Phones,RCA Visys Integrated PBX 8-Line Router,32.700,3,0.0,8.5000
1,1241,CA-2021-107153,2021-09-28,2021-10-03,Standard Class,GZ-14545,George Zrebassa,Corporate,Canada,Montreal,Quebec,H1A,East,OFF-ST-10001321,Office Supplies,Storage,"Decoflex Hanging Personal Folder File, Blue",10.688,2,0.2,3.7408
2,62,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10001025,Furniture,Furnishings,"Eldon Imàge Series Desk Accessories, Clear",24.300,5,0.0,10.4500
3,63,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10002445,Furniture,Furnishings,"DAX Two-Tone Rosewood/Black Document Frame, De...",9.480,1,0.0,3.7900
4,64,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10002960,Furniture,Furnishings,"Eldon 200 Class Desk Accessories, Burgundy",31.400,5,0.0,13.1900


In [111]:
orders_df.shape

(10183, 21)

---

#### Formatting float values to 2 d.p.

In [112]:
orders_df['sales'] = orders_df['sales'].astype(float).round(2)
orders_df['quantity'] = orders_df['quantity'].astype(int).round(2)
orders_df['discount'] = orders_df['discount'].astype(float).round(2)
orders_df['profit'] = orders_df['profit'].astype(float).round(2)

---

#### Final inspection

In [113]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10183 entries, 0 to 10182
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   row_id         10183 non-null  int64         
 1   order_id       10183 non-null  object        
 2   order_date     10183 non-null  datetime64[ns]
 3   ship_date      10183 non-null  datetime64[ns]
 4   ship_mode      10183 non-null  object        
 5   customer_id    10183 non-null  object        
 6   customer_name  10183 non-null  object        
 7   segment        10183 non-null  object        
 8   country        10183 non-null  object        
 9   city           10183 non-null  object        
 10  province       10183 non-null  object        
 11  post_code      10183 non-null  object        
 12  region         10183 non-null  object        
 13  product_id     10183 non-null  object        
 14  category       10183 non-null  object        
 15  sub_category   1018

In [114]:
orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1380,CA-2021-100867,2021-10-19,2021-10-24,Standard Class,EH-14125,Eugene Hildebrand,Home Office,Canada,Vancouver,British Columbia,V6G,West,TEC-PH-10004922,Technology,Phones,RCA Visys Integrated PBX 8-Line Router,32.70,3,0.0,8.50
1,1241,CA-2021-107153,2021-09-28,2021-10-03,Standard Class,GZ-14545,George Zrebassa,Corporate,Canada,Montreal,Quebec,H1A,East,OFF-ST-10001321,Office Supplies,Storage,"Decoflex Hanging Personal Folder File, Blue",10.69,2,0.2,3.74
2,62,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10001025,Furniture,Furnishings,"Eldon Imàge Series Desk Accessories, Clear",24.30,5,0.0,10.45
3,63,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10002445,Furniture,Furnishings,"DAX Two-Tone Rosewood/Black Document Frame, De...",9.48,1,0.0,3.79
4,64,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10002960,Furniture,Furnishings,"Eldon 200 Class Desk Accessories, Burgundy",31.40,5,0.0,13.19


---

### People

#### Overview

In [115]:
people_df.head()

,Regional Manager,Region
0,Sadie Pawthorne,West
1,Chuck Magee,East
2,Roxanne Rodriguez,Central
3,Fred Suzuki,South


In [116]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Regional Manager  4 non-null      object
 1   Region            4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


---

#### Renaming headers

In [117]:
people_df = people_df.rename(columns={
	'Regional Manager': 'manager',
	'Region': 'region'
})

---

#### Final inspection

In [118]:
people_df.tail()

,manager,region
0,Sadie Pawthorne,West
1,Chuck Magee,East
2,Roxanne Rodriguez,Central
3,Fred Suzuki,South


---

### Returns

#### Overview

In [119]:
returns_df.head()

,Returned,Order ID
0,Yes,US-2021-100762
1,Yes,US-2021-100762
2,Yes,US-2021-100762
3,Yes,US-2021-100762
4,Yes,US-2021-100867


In [120]:
returns_df.tail()

,Returned,Order ID
795,Yes,US-2024-147886
796,Yes,US-2024-147998
797,Yes,US-2024-151127
798,Yes,US-2024-155999
799,Yes,US-2024-155999


In [121]:
returns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Returned  800 non-null    object
 1   Order ID  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


---

#### Renaming headers

In [122]:
returns_df.columns = [snake_case(col) for col in returns_df.columns]

---

#### Cleaning


**Checking for null values**

In [123]:
returns_df.isnull().sum()

returned    0
order_id    0
dtype: int64

**Checking for duplicates**

In [124]:
print(returns_df.duplicated().sum())

504


**Show duplicates**

In [125]:
returns_df[returns_df.duplicated()].head(10)

,returned,order_id
1,Yes,US-2021-100762
2,Yes,US-2021-100762
3,Yes,US-2021-100762
6,Yes,US-2021-102652
7,Yes,US-2021-102652
8,Yes,US-2021-102652
11,Yes,US-2021-103744
13,Yes,US-2021-103940
14,Yes,US-2021-103940
15,Yes,US-2021-103940


**Compare ```Returns``` to ```Orders```**

In [126]:
compare_orders_df = orders_df.loc[orders_df['order_id'] == 'US-2021-100762', ['order_id', 'order_date', 'ship_date', 'product_id', 'customer_id']]
compare_orders_df

,order_id,order_date,ship_date,product_id,customer_id
212,US-2021-100762,2021-11-24,2021-11-29,OFF-AR-10000380,NG-18355
213,US-2021-100762,2021-11-24,2021-11-29,OFF-LA-10003930,NG-18355
214,US-2021-100762,2021-11-24,2021-11-29,OFF-PA-10001815,NG-18355
215,US-2021-100762,2021-11-24,2021-11-29,OFF-PA-10004082,NG-18355


In [127]:
# get the number of times each order_id appeared in orders
orders_oid_count = orders_df['order_id'].value_counts().reset_index()
orders_oid_count.columns = ['order_id', 'orders_oid_count']

# get the number of times each order_id appeared in returns
returns_oid_count = returns_df['order_id'].value_counts().reset_index()
returns_oid_count.columns = ['order_id', 'returns_oid_count']

# merge the two df and compare the number of times each order_id appeared in both df
# counting only the order_id that appeared in both df
same_id_entries = orders_oid_count.merge(returns_oid_count, on='order_id', how='inner')

display(same_id_entries)

mismatch = len(same_id_entries[same_id_entries['orders_oid_count'] != same_id_entries['returns_oid_count']])
print(f'Mismatch: {mismatch}')

,order_id,orders_oid_count,returns_oid_count
0,US-2024-100111,14,14
1,US-2023-165330,11,11
2,US-2022-164882,9,9
3,US-2024-161956,8,8
4,US-2024-118087,8,8
...,...,...,...
291,US-2024-161557,1,1
292,US-2022-119907,1,1
293,US-2024-165008,1,1
294,US-2024-165491,1,1


Mismatch: 0


**Conclusion:**

- Each ```order_id``` that appeared in ```Returns``` appeared the same amount of time in ```Orders```. Hence, we can conclude that ```Returns``` is the 'return status' for each product in each order. 

- Hence, we can merge ```Returns``` into ```Orders```.

- However, upon further inspection the ```Returns``` table only contains 'Yes', hence we need to set 'No' to the ```product_id``` that has not been returned.

In [128]:
len(returns_df[returns_df['returned'] != 'Yes'])

0

---

#### Final inspection

In [129]:
returns_df.tail()

,returned,order_id
795,Yes,US-2024-147886
796,Yes,US-2024-147998
797,Yes,US-2024-151127
798,Yes,US-2024-155999
799,Yes,US-2024-155999


---

### Restructuring data

#### Entity Relationship Diagram

<img src="https://github.com/user-attachments/assets/99a8505b-cc9a-4d7d-96db-6410d922f0ee" alt="Image" width="800" height="450">

This step restructures data into different dataframes, which represent different tables in the schema.

#### ```orders``` table:

**Merging ```returns``` with ```orders```**

In [130]:
orders_df['returned'] = orders_df['order_id'].isin(returns_df['order_id']).map({True: 'Yes', False: 'No'})

**Checking the merge**

In [131]:
orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,returned
0,1380,CA-2021-100867,2021-10-19,2021-10-24,Standard Class,EH-14125,Eugene Hildebrand,Home Office,Canada,Vancouver,British Columbia,V6G,West,TEC-PH-10004922,Technology,Phones,RCA Visys Integrated PBX 8-Line Router,32.70,3,0.0,8.50,No
1,1241,CA-2021-107153,2021-09-28,2021-10-03,Standard Class,GZ-14545,George Zrebassa,Corporate,Canada,Montreal,Quebec,H1A,East,OFF-ST-10001321,Office Supplies,Storage,"Decoflex Hanging Personal Folder File, Blue",10.69,2,0.2,3.74,No
2,62,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10001025,Furniture,Furnishings,"Eldon Imàge Series Desk Accessories, Clear",24.30,5,0.0,10.45,No
3,63,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10002445,Furniture,Furnishings,"DAX Two-Tone Rosewood/Black Document Frame, De...",9.48,1,0.0,3.79,No
4,64,CA-2021-115238,2021-01-21,2021-01-24,Standard Class,JW-15220,Jane Waco,Corporate,Canada,Toronto,Ontario,M7A,East,FUR-FU-10002960,Furniture,Furnishings,"Eldon 200 Class Desk Accessories, Burgundy",31.40,5,0.0,13.19,No


In [132]:
orders_df[orders_df['order_id'] == 'US-2021-100762']

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,returned
212,1685,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-AR-10000380,Office Supplies,Art,"Hunt PowerHouse Electric Pencil Sharpener, Blue",151.92,4,0.0,45.58,Yes
213,1689,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-LA-10003930,Office Supplies,Labels,"Dot Matrix Printer Tape Reel Labels, White, 50...",196.62,2,0.0,96.34,Yes
214,1691,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-PA-10001815,Office Supplies,Paper,Xerox 1885,144.12,3,0.0,69.18,Yes
215,1692,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-PA-10004082,Office Supplies,Paper,Adams Telephone Message Book w/Frequently-Call...,15.96,2,0.0,7.98,Yes


In [133]:
orders = orders_df[['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'product_id', 'sales', 'quantity', 'discount', 'profit', 'returned']]

# compare the number of entries in the raw data to the current orders df
print(f'Dimensions: {orders.shape}')
print(f'Must have: ({orders_df.shape[0]}, 11)\n')

orders.info()

Dimensions: (10183, 11)
Must have: (10183, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10183 entries, 0 to 10182
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     10183 non-null  object        
 1   order_date   10183 non-null  datetime64[ns]
 2   ship_date    10183 non-null  datetime64[ns]
 3   ship_mode    10183 non-null  object        
 4   customer_id  10183 non-null  object        
 5   product_id   10183 non-null  object        
 6   sales        10183 non-null  float64       
 7   quantity     10183 non-null  int64         
 8   discount     10183 non-null  float64       
 9   profit       10183 non-null  float64       
 10  returned     10183 non-null  object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 875.2+ KB


---

**```customers``` table**

Note: this table should only contain unique customer data (unique ```customer_id```).

In [134]:
customers = orders_df[['customer_id', 'customer_name', 'segment', 'country', 'city', 'province', 'post_code', 'region']].drop_duplicates(subset=['customer_id'])

# compare the number of unique customer_id in the raw data to the current df
print(f'Dimensions: {customers.shape}')
print(f'Must have: ({len(orders_df['customer_id'].unique())}, 8)\n')

customers.info()

Dimensions: (804, 8)
Must have: (804, 8)

<class 'pandas.core.frame.DataFrame'>
Index: 804 entries, 0 to 10108
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    804 non-null    object
 1   customer_name  804 non-null    object
 2   segment        804 non-null    object
 3   country        804 non-null    object
 4   city           804 non-null    object
 5   province       804 non-null    object
 6   post_code      804 non-null    object
 7   region         804 non-null    object
dtypes: object(8)
memory usage: 56.5+ KB


---

**```products``` table**

In [135]:
products = orders_df[['product_id', 'product_name', 'category', 'sub_category']].drop_duplicates(subset=['product_id'])

print(f'Dimensions: {products.shape}')
print(f'Must have: ({len(orders_df['product_id'].unique())}, 4)\n')

products.info()

Dimensions: (1862, 4)
Must have: (1862, 4)

<class 'pandas.core.frame.DataFrame'>
Index: 1862 entries, 0 to 10101
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    1862 non-null   object
 1   product_name  1862 non-null   object
 2   category      1862 non-null   object
 3   sub_category  1862 non-null   object
dtypes: object(4)
memory usage: 72.7+ KB


---

**```regions``` table**

In [136]:
regions = people_df[['region', 'manager']]
regions.head(len(people_df['region']))

,region,manager
0,West,Sadie Pawthorne
1,East,Chuck Magee
2,Central,Roxanne Rodriguez
3,South,Fred Suzuki


---

## Load

### Connecting to Bigquery

<details>
<summary>Setting up Bigquery connection in Python</summary>

```py
from google.cloud import bigquery as bq
from google.oauth2 import service_account

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'explore29-33756158108f.json'
```
</details>

In [137]:
client = bq.Client()

---

### Creating dataset

<details>
<summary>Create dataset in Bigquery using Python</summary>
<small>

```py
# define the dataset ID
dataset_id = "your_dataset_id"

# create a Dataset reference
dataset_ref = client.dataset(dataset_id)

# create a Dataset object
dataset = bq.Dataset(dataset_ref)

# set the location
dataset.location = "US"

# create dataset
dataset = client.create_dataset(dataset)  # API request
```
</small>
</details>

<details>
<summary>Notes</summary>:

1. Dataset reference
	- This step creates a reference to the dataset.

	- A reference is a lightweight object that points to a resource (in this case, a dataset) in BigQuery.

	- It doesn't contain any metadata or configuration about the dataset itself; it simply identifies the dataset by its ID.

	- Think of it as a pointer or a placeholder for the dataset.

2. Dataset object

	- The Dataset object allows you to configure properties of the dataset, such as:

		- Location: The geographic location where the dataset will be stored (e.g., US, EU, asia-southeast1).

		- Description: A readable description of the dataset.

		- Labels: Key-value pairs for organizing and categorizing datasets.

		- Access Controls: Permissions for who can access the dataset.

	- The Dataset object is used when you want to create, update, or configure a dataset.
</details>

In [138]:
try:
	dataset_ref = client.dataset('superstore')
	dataset = bq.Dataset(dataset_ref)
	dataset.location = 'US'
	dataset = client.create_dataset(dataset)
except Exception:
	print('Dataset already exists')

Dataset already exists


---

### Loading ```orders```

In [139]:
destination_table = 'explore29.superstore.orders'

job_config = bq.LoadJobConfig(
	write_disposition='WRITE_TRUNCATE',
	autodetect=True
)

job = client.load_table_from_dataframe(
	orders,
	destination_table,
	job_config=job_config
)

job.result()

LoadJob<project=explore29, location=US, id=1f7c37d5-ce75-4c84-b4f9-b8d512f26e91>

#### Check ```orders``` data

In [140]:
check = """SELECT * FROM explore29.superstore.orders;"""
query = client.query(check)
query.to_dataframe().head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,returned
0,US-2021-106054,2021-01-06,2021-01-07,First Class,JO-15145,OFF-AR-10002399,12.78,3,0.0,5.24,No
1,US-2021-130092,2021-01-11,2021-01-14,First Class,SV-20365,FUR-FU-10000010,9.94,2,0.0,3.08,No
2,US-2021-149524,2021-01-14,2021-01-15,First Class,BS-11590,FUR-BO-10003433,61.96,4,0.5,-53.29,No
3,US-2021-103366,2021-01-15,2021-01-17,First Class,EH-13990,TEC-AC-10003628,149.95,5,0.0,65.98,No
4,US-2021-146591,2021-01-19,2021-01-20,First Class,TS-21340,FUR-BO-10001972,181.47,5,0.7,-320.60,No


---

### Loading ```customers```

In [141]:
destination_table = 'explore29.superstore.customers'

job_config = bq.LoadJobConfig(
	write_disposition='WRITE_TRUNCATE',
	autodetect=True
)

job = client.load_table_from_dataframe(
	customers,
	destination_table,
	job_config=job_config
)

job.result()

LoadJob<project=explore29, location=US, id=6d245d37-6d57-4bb1-a2fd-982e0cf2fd01>

#### Check ```customers``` data

In [142]:
check = """SELECT * FROM explore29.superstore.customers;"""
query = client.query(check)
query.to_dataframe().head()

,customer_id,customer_name,segment,country,city,province,post_code,region
0,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,01852,East
1,AR-10540,Andy Reiter,Consumer,United States,Woonsocket,Rhode Island,02895,East
2,AM-10705,Anne McFarland,Consumer,United States,Cranston,Rhode Island,02920,East
3,AT-10735,Annie Thurman,Consumer,United States,Bristol,Connecticut,06010,East
4,AA-10480,Andrew Allen,Consumer,United States,Middletown,Connecticut,06457,East


---

### Loading ```products```

In [143]:
destination_table = 'explore29.superstore.products'

job_config = bq.LoadJobConfig(
	write_disposition='WRITE_TRUNCATE',
	autodetect=True
)

job = client.load_table_from_dataframe(
	products,
	destination_table,
	job_config=job_config
)

job.result()

LoadJob<project=explore29, location=US, id=0c4de6f2-90bd-4eea-a9ec-1cd623306bef>

#### Check ```products``` data

In [144]:
check = """SELECT * FROM explore29.superstore.products;"""
query = client.query(check)
query.to_dataframe().head()

,product_id,product_name,category,sub_category
0,TEC-AC-10000474,Kensington Expert Mouse Optical USB Trackball ...,Technology,Accessories
1,TEC-AC-10001314,Case Logic 2.4GHz Wireless Keyboard,Technology,Accessories
2,TEC-AC-10000109,Sony Micro Vault Click 16 GB USB 2.0 Flash Drive,Technology,Accessories
3,TEC-AC-10003441,Kingston Digital DataTraveler 32GB USB 2.0,Technology,Accessories
4,TEC-AC-10001874,Logitech Wireless Anywhere Mouse MX for PC and...,Technology,Accessories


---

### Loading ```regions```

In [145]:
destination_table = 'explore29.superstore.regions'

job_config = bq.LoadJobConfig(
	write_disposition='WRITE_TRUNCATE',
	autodetect=True
)

job = client.load_table_from_dataframe(
	regions,
	destination_table,
	job_config=job_config
)

job.result()

LoadJob<project=explore29, location=US, id=9bde7d56-4a43-4d57-8f30-6b92b05104c6>

#### Check ```regions``` data

In [146]:
check = """SELECT * FROM explore29.superstore.regions;"""
query = client.query(check)
query.to_dataframe().head()

,region,manager
0,West,Sadie Pawthorne
1,East,Chuck Magee
2,Central,Roxanne Rodriguez
3,South,Fred Suzuki
